In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

import csv

In [3]:
dirPath = "data preprocessed\\tf-idf"
dirPath2 = "data preprocessed\\wordvec_6B_50d - TA version" 

In [10]:
train_tf = np.loadtxt(dirPath + '\\train_tf.csv', delimiter=",")
test_tf = np.loadtxt(dirPath + '\\test_tf.csv', delimiter=",")

In [52]:
train_tf.shape, test_tf.shape

((62522, 1312), (8671, 1312))

In [34]:
all_word_unique = open(dirPath + "\\all_word_unique.csv", "r").read().split("\n")[:-1]
len(all_word_unique)

1316

In [8]:
word2vec = pd.read_csv(dirPath2 + "\\wordvec_6B_50d.txt", header = None, delimiter=" ",
                       quoting=csv.QUOTE_NONE, encoding='utf-8')

In [27]:
index = word2vec[0] == all_word_unique[0]
for i in all_word_unique[1:]:
    index |=  word2vec[0] == i
index

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
399970    False
399971    False
399972    False
399973    False
399974    False
399975    False
399976    False
399977    False
399978    False
399979    False
399980    False
399981    False
399982    False
399983    False
399984    False
399985    False
399986    False
399987    False
399988    False
399989    False
399990    False
399991    False
399992    False
399993    False
399994    False
399995    False
399996    False
399997    False
399998    False
399999    False
Name: 0, dtype: bool

In [30]:
word2vec2 = word2vec[index]
word2vec2.shape
word2vec2.head()

(1312, 51)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
65,--,0.49806,1.23820,0.869760,0.002529,-0.562630,-0.387810,-0.47155,-0.957170,-0.123140,...,-0.92491,1.520500,0.81055,0.059992,0.83629,0.17134,-0.526200,-0.15750,0.17409,0.107900
69,people,0.95281,-0.20608,0.556180,-0.463230,0.733540,0.029137,-0.19367,-0.090066,-0.229580,...,-0.95447,0.404820,0.89410,0.652510,0.39858,0.20884,-1.328100,-0.10882,-0.22822,-0.463030
79,time,0.02648,0.33737,0.065667,-0.116090,0.416510,-0.211420,-0.69582,0.282200,-0.360770,...,-0.10344,0.006977,-0.17328,0.355360,-0.46369,0.15285,0.414750,-0.33980,-0.23043,0.190690
115,city,0.43945,0.43268,-0.366540,0.277810,0.062932,-0.802010,-0.93041,0.016388,-0.550300,...,-0.66711,0.607990,0.23787,-0.653780,-0.70550,0.51650,-1.078000,-0.71524,0.48402,-0.325600
122,day,0.11626,0.53897,-0.395140,-0.260270,0.577060,-0.791980,-0.88374,0.301190,0.082896,...,-0.87920,0.261340,-0.32132,-0.126520,0.25573,0.32818,0.024073,-0.39062,-0.10885,0.084513


In [35]:
set(all_word_unique).difference(set(list(word2vec2[0])))

{'delish', 'guac', 'phx', 'yelpers'}

In [37]:
for i in {'delish', 'guac', 'phx', 'yelpers'}:
    print(all_word_unique.index(i))

626
1195
1045
935


TA给的数据少了这几个词频大于500的词。那么为了兼容，需要去除这4个词，TF矩阵要修改。

In [43]:
train_tf = np.delete(train_tf, [626, 1195, 1045, 935], axis=1)
train_tf.shape

(62522, 1312)

In [44]:
test_tf = np.delete(test_tf, [626, 1195, 1045, 935], axis=1)
test_tf.shape

(8671, 1312)

In [36]:
all_word_unique_ta= list(word2vec2[0])
len(all_word_unique_ta)

1312

In [45]:
word2vec2.shape
word2vec2.drop([0], axis=1, inplace=True)
word2vec2.shape

(1312, 51)

E:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(1312, 50)

In [46]:
word2vec_arr = arr(word2vec2.values)
word2vec_arr.shape
word2vec_arr[0]

(1312, 50)

array([  4.98060000e-01,   1.23820000e+00,   8.69760000e-01,
         2.52930000e-03,  -5.62630000e-01,  -3.87810000e-01,
        -4.71550000e-01,  -9.57170000e-01,  -1.23140000e-01,
         6.32620000e-01,  -7.33750000e-01,  -6.34570000e-02,
        -6.64770000e-01,   6.80960000e-01,   6.35950000e-01,
         8.17480000e-01,   1.51830000e-01,  -9.75820000e-01,
         2.36740000e-01,   1.77670000e-01,  -4.17310000e-01,
         1.25860000e-01,   8.80720000e-01,  -4.94020000e-02,
        -8.30490000e-01,  -2.42830000e+00,  -1.30540000e+00,
         1.65000000e-01,  -5.60370000e-02,  -1.19530000e-02,
         3.01080000e+00,   8.93620000e-01,  -4.98190000e-01,
         5.65450000e-01,   9.55350000e-01,  -6.85070000e-01,
        -1.45200000e-01,   1.40260000e-01,   2.28410000e-01,
         1.97700000e-01,  -9.24910000e-01,   1.52050000e+00,
         8.10550000e-01,   5.99920000e-02,   8.36290000e-01,
         1.71340000e-01,  -5.26200000e-01,  -1.57500000e-01,
         1.74090000e-01,

In [49]:
train_w2v = np.dot(train_tf, word2vec_arr)
test_w2v = np.dot(test_tf, word2vec_arr)

train_w2v.shape, test_w2v.shape

((62522, 50), (8671, 50))

In [50]:
#划分比例
splitRate = 0.3
#划分的数目
splitNum = int(train_w2v.shape[0]*splitRate) 
#得到 训练集 和验证集
trainSet_w2v = train_w2v[:-splitNum]
validateSet_w2v = train_w2v[-splitNum:]

trainSet_w2v.shape, validateSet_w2v.shape

((43766, 50), (18756, 50))

In [51]:
dirPath2 = "data preprocessed\\wordvec_6B_50d - TA version" 
np.savetxt(dirPath2 + '\\train.csv', trainSet_w2v, delimiter=",")
np.savetxt(dirPath2 + '\\validate.csv', validateSet_w2v, delimiter=",")
np.savetxt(dirPath2 + '\\test.csv', test_w2v, delimiter=",")